In [24]:
import sys
import os
import pandas as pd

# Get the directory of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, ".."))

run_results = pd.read_parquet(os.path.abspath(
    os.path.join(notebook_dir, "../../cache/parquet/run_results.parquet")
))

print(run_results)

      season  week     gameId gameDisplay           gameDate homeTeamId  \
0       2024     6  401671820  IND vs TEN  2024-10-13T17:00Z         10   
1       2024     6  401671819   SF vs SEA  2024-10-11T00:15Z         26   
2       2024     6  401671714  CLE vs PHI  2024-10-13T17:00Z         21   
3       2024     6  401671657   PIT vs LV  2024-10-13T20:05Z         13   
4       2024     6  401671684  BUF vs NYJ  2024-10-15T00:15Z         20   
...      ...   ...        ...         ...                ...        ...   
1784    2015     3  400791491  ATL vs DAL  2015-09-27T17:00Z          6   
1785    2015     3  400791500  OAK vs CLE  2015-09-27T17:00Z          5   
1786    2015     3  400791557   NO vs CAR  2015-09-27T17:00Z         29   
1787    2015     3  400791546  CIN vs BAL  2015-09-27T17:00Z         33   
1788    2015     3  400791597   SF vs ARI  2015-09-27T20:05Z         22   

     homeTeamDisplay  homeScore awayTeamId awayTeamDisplay  awayScore  \
0                TEN      

In [26]:
def process_alignment(df: pd.DataFrame):
    res = df[
        ((df['rating'] > 0) & (df['stats'] > 0)) |
        ((df['rating'] < 0) & (df['stats'] < 0))
    ]

    align = len(res.index) / len(df.index)

    print('aligned', align)

print('--sanity--')
total = len(run_results.index)

print('total', len(run_results.index))
print('home selected', len(run_results[run_results['grade'] == 2].index) / total)
print('away selected', len(run_results[run_results['grade'] == -2].index) / total)
print('unknown', len(run_results[run_results['grade'] == 0].index) / total)

process_alignment(run_results)

chosen = run_results[
    (run_results['grade'] == 2) | (run_results['grade'] == -2)
]

correct = chosen[
    ((chosen['grade'] == 2) & (chosen['expected'] == True)) |
    ((chosen['grade'] == -2) & (chosen['expected'] == False))
]

incorrect = chosen[~chosen.index.isin(correct.index)]

print('accuracy', len(correct.index) / len(chosen.index))

print('rating ->', chosen['rating'].abs().mean(), 'stats ->', chosen['stats'].abs().mean())
column = incorrect['rating'].abs()
print('rating - incorrect', column.min(), column.mean(), column.max())
column = correct['rating'].abs()
print('rating - correct', column.min(), column.mean(), column.max())

column = incorrect['stats'].abs()
print('stats - incorrect', column.min(), column.mean(), column.max())
column = correct['stats'].abs()
print('stats - correct', column.min(), column.mean(), column.max())

--sanity--
total 1789
home selected 0.2582448295136948
away selected 0.26774734488541085
unknown 0.47400782560089433
aligned 0.5259921743991056
Empty DataFrame
Columns: [season, week, gameDisplay, rating, stats]
Index: []
accuracy 0.61211477151966
rating -> 79.58858890360078 stats -> 92.83089496148301
rating - incorrect 0.5225472155680109 72.99384720044515 387.82837453230945
rating - correct 0.005083507365156947 83.76754848980184 500.6525509207397
stats - incorrect 0.08643501266226394 90.83273063579537 374.3460023778987
stats - correct 0.05323646704891871 94.09709284147606 380.9521518688855


In [30]:
sub_select = chosen[
    (abs(chosen['stats']) > 95) & (abs(chosen['rating']) > 85)
]

sub_correct = sub_select[
    ((sub_select['grade'] == 2) & (sub_select['expected'] == True)) |
    ((sub_select['grade'] == -2) & (sub_select['expected'] == False))
]

print('available', total)
print('correct', len(sub_correct.index), 'chosen', len(sub_select.index))
print('accuracy', len(sub_correct.index) / len(sub_select.index))

available 1789
correct 109 chosen 157
accuracy 0.6942675159235668
